In [13]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [14]:
# Load data and select trainging hour and test hour
trainhour = 5
testhour = 7
df = pd.read_csv('series.csv')
cols5 = ['21', '22', '23', '24', '25']
cols10 = ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
df['sum5'] = df[cols5].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [15]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["sum10"].apply(setlabels)

In [16]:
# Get X, y for model
X = data.iloc[:,8:28].to_numpy()

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = to_categorical(y_encoded)

In [17]:
# Split data into training and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [18]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 12,503
Trainable params: 12,503
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Fit model
model.fit(X_train, y_train, epochs=5, shuffle=True, verbose=0)

In [20]:
# Model loss and Accuray
model_loss, model_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.17681715957925281, Accuracy: 0.9582903981208801
Normal Neural Network - Loss: 0.2296009317239944, Accuracy: 0.9515963196754456


In [21]:
# Pull out validation data and make predicts
formodel = testdata.iloc[:,8:28].to_numpy()
testdata['class'] = label_encoder.inverse_transform(model.predict_classes(formodel))
testdata = testdata.drop(testdata[(testdata['class'] == 0)].index)

In [22]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,28+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp < 2):
              break  
        else:
            if(temp < -2):
              break
        
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = -temp
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: -45.29999999999995


In [23]:
#testdata.to_csv('binarytest.csv')